In [22]:
import pandas as pd
import pytz

In [23]:
clima = pd.read_csv('../Bases/Raw/INMET_SE_SP_A701_SAO PAULO - MIRANTE_01-01-2019_A_31-12-2019.CSV', sep=';', header=8, encoding='utf-8')
clima

,Data,Hora UTC,"PRECIPITACAO TOTAL, HORARIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSAO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSAO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIRECAO HORARIA (gr) ((gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2019/01/01,0000 UTC,0,"925,6","925,6","925,2",NaN,"22,8","16,9",23,"22,7","17,3","16,8",71,68,69,123,7,"2,6",NaN
1,2019/01/01,0100 UTC,0,926,926,"925,6",NaN,"22,3","16,8","22,8","22,3","16,9","16,4",71,68,71,107,"6,8","2,5",NaN
2,2019/01/01,0200 UTC,0,"925,5",926,"925,5",NaN,"22,1",17,"22,3","22,1","17,1","16,8",73,71,73,109,"6,5","2,1",NaN
3,2019/01/01,0300 UTC,0,"924,7","925,5","924,7",NaN,"21,6","17,1","22,1","21,6","17,2",17,76,73,76,101,"5,8","1,6",NaN
4,2019/01/01,0400 UTC,0,"924,4","924,7","924,4",NaN,"21,6","17,1","21,6","21,5","17,2","17,1",76,75,76,107,"5,1","2,2",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019/12/31,1900 UTC,",2","920,4","920,7","920,1","575,9","26,2","15,9","27,8","25,2","18,8","15,5",63,52,53,74,"9,9",",6",NaN
8756,2019/12/31,2000 UTC,0,"920,4","920,7","920,3","600,3","24,9","18,1","26,6","24,8","18,4","15,7",66,52,66,37,"4,7","1,4",NaN
8757,2019/12/31,2100 UTC,0,920,"920,4","919,9","922,5","26,5","18,5","26,5","24,9","18,6","17,6",67,59,62,137,"4,3","1,5",NaN
8758,2019/12/31,2200 UTC,0,"920,5","920,5",920,"83,9",25,"18,7","26,5",25,"18,8","18,2",68,61,68,164,"4,1","1,7",NaN


In [24]:
#Pegando apenas colunas com informações de data, precipitação e temperatudo. E o intervalo de tempo apenas dos nossos dados
clima = clima[['Data','Hora UTC','PRECIPITACAO TOTAL, HORARIO (mm)','TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)','TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)']]
clima = clima.loc[(clima['Data'] >= '2019/06/01') & (clima['Data'] <= '2019/10/10')]

In [25]:
#Convertendo a data e a hora em um datetime
clima['Datetime'] = pd.to_datetime(clima['Data']+' '+clima['Hora UTC'])
clima['Datetime'] = clima['Datetime'].dt.tz_convert(pytz.timezone('America/Sao_Paulo'))
clima['Datetime'] = clima['Datetime'].dt.tz_localize(None)

In [26]:
# Tirando a média das temperaturas mínima e máxima
clima["Temperatura Media"] = (clima["TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float) + clima["TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)"].str.replace(",", ".").astype(float))/2

In [27]:
#Transformando a precipitação em um valor categorico - Usamos apenas 3 Sem, Fraco e Forte pois com mais categorias algumas ficavam pouco representativas
clima["PRECIPITACAO TOTAL, HORARIO (mm)"] = clima["PRECIPITACAO TOTAL, HORARIO (mm)"].str.replace(",", ".").astype(float)
clima['Descricao Chuva'] = clima['PRECIPITACAO TOTAL, HORARIO (mm)'].apply(lambda x: 1 if (x > 0) & (x <= 2.5) else \
2 if (x > 2.5) & (x <= 10) else \
3 if (x > 10) else 0)

In [28]:
#Transformando a Temperatura em um valor categorico - Usamos apenas 2 Frio ou Quente pois com mais categorias algumas ficavam pouco representativas
clima['Calor'] = clima['Temperatura Media'].apply(lambda x: '1' if (x > 16) & (x <= 25) else \
'2' if (x > 25) else '0')

In [29]:
#Dropando as colunas desnecessárias
clima.drop(["Data", "Hora UTC", "TEMPERATURA MAXIMA NA HORA ANT. (AUT) (°C)", "TEMPERATURA MINIMA NA HORA ANT. (AUT) (°C)",'PRECIPITACAO TOTAL, HORARIO (mm)','Temperatura Media'], axis=1, inplace=True)

In [31]:
clima.to_csv('clima.csv',index=False)